##### 先观察全集的情况

In [1]:
import json

file = 'data/standard section/sec_to_name.json'
with open(file, 'r') as f:
    datas = json.load(f)

len(datas)

1624132

In [2]:
sec_name_dict = {}
for data in datas:
    if datas[data] in sec_name_dict:
        sec_name_dict[datas[data]] += 1
    else:
        sec_name_dict[datas[data]] = 1

sec_name_dict = sorted(sec_name_dict.items(), key=lambda x:x[1], reverse=True)

json_data = json.dumps(sec_name_dict, indent=4)
with open('data/standard section/sec_name_dict.json', 'w') as f:
    f.write(json_data)

In [3]:
# 去除掉数字和特殊符号
import re

sec_name_dict = {}
for data in datas:
    name = datas[data]
    try:
        name = re.sub(r'[^a-zA-Z\s]', '', name).strip()
        if name in sec_name_dict:
            sec_name_dict[name] += 1
        else:
            sec_name_dict[name] = 1
    except:
        continue

sec_name_dict = sorted(sec_name_dict.items(), key=lambda x:x[1], reverse=True)
print(len(sec_name_dict))

json_data = json.dumps(sec_name_dict, indent=4)
with open('data/standard section/sec_name_dict_remove_number.json', 'w') as f:
    f.write(json_data)

46224


In [14]:
standard_section_names = [
    'introduction',
    'result',
    'method',
    'discussion',
    'conclusion',
    'supporting information',
    'literature review'
]

sec_name_dict = {}
with open('data/standard section/sec_name_dict_remove_number.json', 'r') as f:
    datas = json.load(f)

for data in datas:
    name = data[0].lower()
    flag = False
    for stan_name in standard_section_names:
        if stan_name in name:
            flag = True
            if stan_name in sec_name_dict:
                sec_name_dict[stan_name] += data[1]
            else:
                sec_name_dict[stan_name] = data[1]
            break
    if not flag:
        if name in sec_name_dict:
            sec_name_dict[name] += data[1]
        else:
            sec_name_dict[name] = data[1]

sec_name_dict_item = sorted(sec_name_dict.items(), key=lambda x:x[1], reverse=True)
print(len(sec_name_dict_item))

json_data = json.dumps(sec_name_dict_item, indent=4)
with open('data/standard section/sec_name_dict_simple_standard.json', 'w') as f:
    f.write(json_data)

39105


In [15]:
num = 0
for name in standard_section_names:
    num += sec_name_dict[name]

all_record = 1624132
print(num/all_record)

0.9424812761524309


In [16]:
stan_to_unstan = {
    'introduction':['background',],
    'result':['finding', 'analyse', 'analysis'],
    'method':['experiment','data','material','design','model','approach'],
    'discussion':['implication'],
    'conclusion':['limitation','future work','future','application'],
    'literature review':['related work', 'literature']
    # 'supporting information':[]    
}

unstan_to_stan = {}
for k, v in stan_to_unstan.items():
    for i in v:
        unstan_to_stan[i] = k

unstan_to_stan

{'background': 'introduction',
 'finding': 'result',
 'analyse': 'result',
 'analysis': 'result',
 'experiment': 'method',
 'data': 'method',
 'material': 'method',
 'design': 'method',
 'model': 'method',
 'approach': 'method',
 'implication': 'discussion',
 'limitation': 'conclusion',
 'future work': 'conclusion',
 'future': 'conclusion',
 'application': 'conclusion',
 'related work': 'literature review',
 'literature': 'literature review'}

In [17]:
further_sec_name_dict = {}
with open('data/standard section/sec_name_dict_simple_standard.json', 'r') as f:
    datas = json.load(f)

for data in datas:
    name = data[0].lower()
    flag = False
    for unstan_name in unstan_to_stan.keys():
        if unstan_name in name:
            flag = True
            stan_name = unstan_to_stan[unstan_name]
            if stan_name in further_sec_name_dict:
                further_sec_name_dict[stan_name] += sec_name_dict[name]
            else:
                further_sec_name_dict[stan_name] = sec_name_dict[name]
            break
    if not flag:
        if name in further_sec_name_dict:
            further_sec_name_dict[name] += data[1]
        else:
            further_sec_name_dict[name] = data[1]

sec_name_dict_item = sorted(further_sec_name_dict.items(), key=lambda x:x[1], reverse=True)
print(len(sec_name_dict_item))

json_data = json.dumps(sec_name_dict_item, indent=4)
with open('data/standard section/sec_name_dict_further_standard.json', 'w') as f:
    f.write(json_data)

28339


In [18]:
num = 0
for name in standard_section_names:
    num += further_sec_name_dict[name]

all_record = 1624132
print(num/all_record)

0.9635817778357917


##### 对原始数据进行替换

In [ ]:
import json

file = 'data/standard section/sec_to_name.json'
with open(file, 'r') as f:
    datas = json.load(f)

raw_sections = list(set(datas.values()))

55165

In [ ]:
stan_to_unstan = {
    'introduction':['introduction', 'background',],
    'result':['result', 'finding', 'analyse', 'analysis'],
    'method':['method', 'experiment','data','material','design','model','approach'],
    'discussion':['discussion', 'implication'],
    'conclusion':['conclusion', 'limitation','future work','future','application'],
    'literature review':['literature review', 'related work', 'literature'],
    'supporting information':['supporting information']    
}

unstan_to_stan = {}
for k, v in stan_to_unstan.items():
    for i in v:
        unstan_to_stan[i] = k

In [12]:
from tqdm import tqdm
import re

sec_raw_to_new = {}
for section in tqdm(raw_sections):
    raw_section = section
    try:
        section = re.sub(r'[^a-zA-Z\s]', '', section).strip().lower()
        for unstan_name in unstan_to_stan:
            if unstan_name in section:
                section = unstan_to_stan[unstan_name]
                break
    except:
        section = 'None Value'
    sec_raw_to_new[raw_section] = section

100%|██████████| 55165/55165 [00:00<00:00, 193480.09it/s]


In [14]:
standard_datas = {}
for data in datas:
    raw_section = datas[data]
    standard_datas[data] = sec_raw_to_new[raw_section]

json_data = json.dumps(standard_datas, indent=4)
with open('data/standard section/sec_to_name_standard.json', 'w') as f:
    f.write(json_data)